In [1]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (osxkeychain).

In [4]:
# Enable weights and biases
import os
os.environ['WANDB_DISABLED']="true"

In [10]:
# compute_dtype = getattr(torch, "float16")
# bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type='nf4',
#         bnb_4bit_compute_dtype=compute_dtype,
#         bnb_4bit_use_double_quant=False,
#     )

In [8]:
# Load the dataset
dataset = load_dataset("csv", data_files={"train": "../data/processed/phishing_train_llm.csv", "test": "../data/processed/phishing_test_llm.csv"})
# Load the model Phi-2
model_name='microsoft/phi-2'
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
# model = AutoModelForCausalLM(model_name,device_map="auto")


Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.08s/it]


In [14]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [24]:
# Test the Model with Zero Shot Inferencing
from transformers import set_seed
set_seed(42)

index = 20

body = dataset['test'][0]['body']
label = dataset['test'][0]['label']
prompt = f"""
Instruct: Classify the following body as "Phishing Email" or "Safe Email".
Body: {body}
Response:\n"""

inputs = tokenizer(prompt, return_tensors="pt")
res = model.generate(**inputs, max_new_tokens=90)
translated_text = tokenizer.batch_decode(res, skip_special_tokens=True)[0]
print(translated_text)
print(label)
# print(model.can_generate())
# for i in range(index):
#     body = dataset['test'][i]['body']
#     label = dataset['test'][i]['label']
#     prompt = f"""
#                 Instruct: Classify the following body as "Phishing Email" or "Safe Email".
#                 Body: {body}
#                 Response:\n
#             """
#     # Tokenize the prompt
#     inputs = tokenizer(prompt, return_tensors="pt")
#     # Generate the response
#     res = model.generate(**inputs, max_new_tokens=10)
#     print(res)
#     print(label)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Instruct: Classify the following body as "Phishing Email" or "Safe Email".
Body: Did you know? You can refinance up to 1 time per year.
Stop fighting for lenders let them fight for you! 
Make them work for your business by giving you the lowest rates around!
Please fill out 30 second insta-quote and monthly savings calculation
http://skeletosyard.com
Response:
Phishing Email

Phishing Email


In [ ]:
# Test the model with few shot inferencing
